In [ ]:
%load_ext autoreload
%autoreload 2

# Prepare dataset using pygrib

The purpose of this notebook is to prepare a statistical downscaling dataset
using pygrib and XArray. Notebook *018-prepare-dataset.ipynb* showed that 
it is not really practical to do so with XArray and CFGrib only. Indeed,
opening a large collection of grib files is very long with CFGrib, and 
the API limitations on filtering the fields to have only consistent
datacubes force us to open the files multiple times.

Our overall strategy here is going to be:
1. Extract relevant data from grib to NetCDF using PyGrib.
2. The NetCDF will not be CF compliant, it will only be an
   intermediate file format.
3. Open the NetCDF files using XArray, and then perform the
   interpolation.
  
Our hope is that XArray will be much happier playing with netCDF files 
than Grib files, and that we will be able to open and interpolate the 
data only once using XArray and dropping strict CF convention compliance.

# 1. Extract relevant data using pygrib

In [ ]:
import dask
import dask.distributed
import dask_jobqueue
import datetime
import multiprocessing
import netCDF4
import os
import pathlib
import pygrib
import xarray as xr

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
GDPS_DIR = DATA_DIR / '2021-02-08-one-month-sample/'
NETCDF_DIR = DATA_DIR / 'hdd_scratch/smc01/2021-02-09-one-month-single-file'

TO_EXTRACT = [
    'prmsl',
    'hpbl',
    'prate',
    '2t',
    '2d',
    '2r',
    '10si',
    '10wdir',
]

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=[
        'source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=10)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
gdps_path = pathlib.Path(GDPS_DIR)
output_path = pathlib.Path(NETCDF_DIR)

In [ ]:
output_path.mkdir(exist_ok=True)

In [ ]:
grib_files = sorted(list(gdps_path.glob('*.grib2')))

In [ ]:
grib_files[0:10]

### Figure out how many dates and steps there are

In [ ]:
pass_strings = [g.stem[22:32] for g in grib_files]

In [ ]:
unique_passes = sorted(list(set(pass_strings)))

In [ ]:
unique_passes

In [ ]:
n_pass = len(unique_passes)

In [ ]:
step_strings = [g.stem[-4:] for g in grib_files]

In [ ]:
step_strings[0:10]

In [ ]:
unique_steps = sorted(list(set(step_strings)))

In [ ]:
n_step = len(unique_steps)

In [ ]:
n_step

In [ ]:
grib_files[0].stem

In [ ]:
target_file_name = grib_files[0].stem + '_filtered.nc'
target_file_path = output_path / target_file_name

In [ ]:
target_file_path

In [ ]:
lambdas = [
    lambda x: x.shortName == 'st' and x.typeOfLevel == 'surface',
]

In [ ]:
def compound_lambda(message):
    for l in lambdas:
        if l(message):
            return True
        
    return False

In [ ]:
def pass_to_datetime(pass_string):
    print(pass_string)
    return datetime.datetime(
        int(pass_string[:4]),
        int(pass_string[4:6]),
        int(pass_string[6:8]),
        int(pass_string[8:10])
    )

In [ ]:
def prepare_netcdf_file(output_file, date, step):
    time_units = 'seconds since 1970-01-01 00:00:00.0'

    output_file.createDimension('latitude', size=751)
    output_file.createDimension('longitude', size=1500)
    output_file.createDimension('time', size=1)
    output_file.createDimension('step', size=1)

    step_var = output_file.createVariable('step', 'i4', dimensions=('step'))
    step_var[0] = step

    time_var = output_file.createVariable('time', 'f8', dimensions=('time'))
    float_date = netCDF4.date2num(date, time_units)
    time_var[0] = float_date
    time_var.units = time_units

    variables = {}
    for short_name in TO_EXTRACT:
        variables[short_name] = output_file.createVariable(
            short_name, 'f4', dimensions=('time', 'step', 'latitude', 'longitude'),
            zlib=True) 
        
    return variables

In [ ]:
def add_latlon_to_file(output_file, lat, lon):
    lat_var = output_file.createVariable('latitude', 'f4', dimensions=('latitude'))
    lat_var[:] = lat

    lon_var = output_file.createVariable('longitude', 'f4', dimensions=('longitude'))
    lon_var[:] = lon

In [ ]:
def handle_one_file(input_dir, pass_string, step_string, output_dir):
    print(pass_string)
    target_file_path = output_dir / (f'gdps_{pass_string}_{step_string}.nc')
    
    if target_file_path.is_file():
        print(f'Skipping {pass_string} because output file already exists')
    
    output_file = netCDF4.Dataset(str(target_file_path), 'w')
    date = pass_to_datetime(pass_string)
    
    step = int(step_string[1:])
    
    variables = prepare_netcdf_file(output_file, date, step)

    print(pass_string, step_string)
    grib_file_name = f'CMC_glb_latlon.24x.24_{pass_string}_{step_string}.grib2'
    grib_file_path = input_dir / grib_file_name

    grib_file = pygrib.open(str(grib_file_path))

    lat, lon = grib_file[1].latlons()
    lat = lat[:,0]
    lon = lon[0]

    add_latlon_to_file(output_file, lat, lon)

    for message in grib_file:
        if message.shortName in TO_EXTRACT:
            var = variables[message.shortName]
            var[0,0,:] = message.values

    output_file.close()

### Manually

In [ ]:
for p in unique_passes[:1]:
    for s in unique_steps[:1]:
        handle_one_file(gdps_path, p, s, output_path)

### With multiprocessing

In [ ]:
with multiprocessing.Pool(processes=8) as pool:
    pool.starmap(handle_one_pass, [(gdps_path, p, output_path) for p in unique_passes])

### With dask

In [ ]:
handle_one_file_delayed = dask.delayed(handle_one_file)

In [ ]:
delayeds = [handle_one_file_delayed(gdps_path, p, s, output_path) for p in unique_passes for s in unique_steps]

In [ ]:
delayeds[0:10]

In [ ]:
len(delayeds)

In [ ]:
dask.compute(*delayeds)

In [ ]:
target_file_name = grib_files[0].stem + '_filtered.nc'
target_file_path = output_path / target_file_name

grib_file.seek(0)
for grib_file_path in grib_files:
    grib_file = pygrib.open(str(grib_file_path))
    for message in grib_file:
        if message.shortName in to_extract or compound_lambda(message):
            print(message)
            
    grib_file.close()

In [ ]:
lat, lon = grib_file[1].latlons()
lat = lat[:,0]
lon = lon[0]

In [ ]:
target_file_path.unlink()

In [ ]:
root = netCDF4.Dataset(str(target_file_path), 'w')

In [ ]:
lon[0]

In [ ]:
root

In [ ]:
root.createDimension('latitude', size=lat.shape[0])

In [ ]:
root.createDimension('longitude', size=lon.shape[0])

In [ ]:
root.createDimension('time', size=0)

In [ ]:
root.createDimension('step', size=81)

In [ ]:
lat_var = root.createVariable('latitude', 'f4', dimensions=('latitude'))

In [ ]:
lat_var[:] = lat

In [ ]:
lon_var = root.createVariable('longitude', 'f4', dimensions=('longitude'))

In [ ]:
lon_var[:] = lon

In [ ]:
grib_file.seek(0)
for message in grib_file:
    if message.shortName in to_extract or compound_lambda(message):
        var = root.createVariable(message.shortName, 'f4', dimensions=('latitude', 'longitude'))
        var[:] = message.values

In [ ]:
root

In [ ]:
root.close()

## Open using Xarray

In [ ]:
def nest_filenames(files):
    passes = {}
    for f in files:
        pass_name = f.stem[5:15]
        
        pass_list = passes.get(pass_name, [])
        pass_list.append(f)
        passes[pass_name] = pass_list
        
    sorted_passes = sorted(passes.keys())
        
    return [passes[k] for k in sorted_passes]

In [ ]:
nc_filenames = sorted(list(output_path.glob('*.nc')))

In [ ]:
nc_filenames[1:10]

In [ ]:
nested_nc = nest_filenames(nc_filenames)

In [ ]:
gdps = xr.open_mfdataset(
    nested_nc, concat_dim=['time', 'step'], 
    combine='nested', parallel=True, compat='no_conflicts')

In [ ]:
gdps

In [ ]:
gdps.sel(time=datetime.datetime(2020,7,22,12), step=15)['2t'].plot()

In [ ]:
gdps.mean(dim='time').sel(step=0)['2t'].plot()